# Client Test

In [12]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'micropython')))
 
import client
from collections import OrderedDict

### Start client

In [13]:
the_client = client.Client()
the_client.start()

My name is Client_366

[Connected: ('192.168.0.105', 9662)]
Sending 313 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-02-02 01:14:27.427100'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2017-02-02 01:14:27.427100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

[Listen to messages]


### Prepare messages

In [14]:
messages = OrderedDict()

messages['read_GPIOs'] = {'message_type': 'command',
                          'command': 'read GPIOs',
                          'kwargs': {'pins': (5, 12, 13, 14, 15, 16)},
                          'need_result': True}

messages['blink_led'] = {'message_type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 10, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}} 


Data received: 409 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:14:27.427100'), ('message_id', '2017-02-02 01:14:27.495100'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'Client_366': "('192.168.0.105', 53720)", 'NodeMCU_1dsc000': "('192.168.0.133', 18714)", 'NodeMCU_f1d30800': "('192.168.0.116', 23664)"}), ('sender', 'Hub'), ('time_stamp', '2017-02-02 01:14:27.495100')])



In [15]:
# messages['write_GPIOs'] = {'message_type': 'command',
                           # 'command': 'write GPIOs',
                           # 'kwargs': {'pins_and_values': [(2, 0), (2, 1), (2, 0),]}} 

# messages['test eval'] = {'message_type': 'eval',
                         # 'to_evaluate': '2+3',
                         # 'need_result': True}                                   

# messages['test exec'] = {'message_type': 'exec',
                         # 'to_exec': 'print("Testing exec !")'}

# with open('script_to_deploy.py') as f:
    # script = f.read()        
# messages['test upload script'] = {'message_type': 'script', 
                                  # 'script': script} 

### Ask Hub for a list of connected nodes

In [16]:
while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

In [17]:
message = {'message_type': 'command',
           'command': 'list connections by name',
           'need_result': True}     

_, asynch_result = the_client.request('Hub', message) 

try:
    remote_nodes = sorted(list(asynch_result.get().keys()))

    print ('\n[____________ Connected nodes ____________]\n')        
    print('\nConnected nodes:\n{}\n'.format(remote_nodes)) 
    
except Exception as e:
    print(e)

Sending 284 bytes
Message:
OrderedDict([('command', 'list connections by name'), ('correlation_id', '2017-02-02 01:14:28.103100'), ('message_id', '2017-02-02 01:14:28.103100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 409 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:14:28.103100'), ('message_id', '2017-02-02 01:14:28.162100'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'Hub'), ('result', {'Client_366': "('192.168.0.105', 53720)", 'NodeMCU_1dsc000': "('192.168.0.133', 18714)", 'NodeMCU_f1d30800': "('192.168.0.116', 23664)"}), ('sender', 'Hub'), ('time_stamp', '2017-02-02 01:14:28.162100')])


[____________ Connected nodes ____________]


Connected nodes:
['Client_366', 'NodeMCU_1dsc000', 'NodeMCU_f1d30800']



### Read one GPIO pin

In [18]:
for remote_node in remote_nodes:
    _, result = the_client.request(remote_node, messages['read_GPIOs'] ) 
    print('\nGPIO status for {}: {}\n'.format(remote_node, result.get()))

Sending 322 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-02 01:14:28.298100'), ('kwargs', {'pins': (5, 12, 13, 14, 15, 16)}), ('message_id', '2017-02-02 01:14:28.298100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 366 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-02 01:14:28.298100'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2017-02-02 01:14:28.298100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366'), ('time_stamp', '2017-02-02 01:14:28.384100')])


Processed result:
OrderedDict([('correlation_id', '2017-02-02 01:14:28.298100'), ('message_id', '2017-02-02 01:14:28.450100'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('result', 'Not applicable.'), ('sender', 

### Blink each node

In [19]:
# remote_node = 'NodeMCU_|edca00'
# the_client.request(remote_node, messages['blink_led']) 

# for remote_node in remote_nodes:
#     the_client.request(remote_node, messages['blink_led']) 

### Send out messages and get asynchonous results

In [20]:
print ('\n[______________ Sending messages ______________]\n')
                
results = []

## send out the messages
for message in messages.values():
    for remote_node in remote_nodes:
        if remote_node != the_client.node.worker.name:  # exclude client self
            formatted_message, asynch_result = the_client.request(remote_node, message) 
            results.append((formatted_message, asynch_result))


[______________ Sending messages ______________]

Sending 327 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-02 01:14:29.502100'), ('kwargs', {'pins': (5, 12, 13, 14, 15, 16)}), ('message_id', '2017-02-02 01:14:29.502100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 328 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-02 01:14:29.555100'), ('kwargs', {'pins': (5, 12, 13, 14, 15, 16)}), ('message_id', '2017-02-02 01:14:29.555100'), ('message_type', 'command'), ('need_result', True), ('receiver', 'NodeMCU_f1d30800'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 342 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-02-02 01:14:29.610100'), ('kwargs', {'times': 10, 'off_seconds': 0.1, 'on_seconds': 0.1, 'forever': False}), ('message_id', '2017-02-02 01:14:29.610100'), 

### Actually get the results

In [21]:
# collect and print results        
print('\n[_________ Wait few seconds for reply _________]\n')

for (message, result) in results:
    try:
        if message.get('need_result'):
            print('\n[Result for request]:\n___Request___:\n{0}\n___Result____:\n{1}\n'.format(message, result.get() if result else None))
    except Exception as e:
        print('\n[{}]\nMessage:\n{}'.format(e, message))


[_________ Wait few seconds for reply _________]


[Result for request]:
___Request___:
{'correlation_id': '2017-02-02 01:14:29.502100', 'reply_to': 'Client_366', 'sender': 'Client_366', 'kwargs': {'pins': (5, 12, 13, 14, 15, 16)}, 'receiver': 'NodeMCU_1dsc000', 'message_type': 'command', 'need_result': True, 'message_id': '2017-02-02 01:14:29.502100', 'command': 'read GPIOs'}
___Result____:
[[5, 1], [12, 1], [13, 1], [14, 1], [15, 0], [16, 0]]


Data received: 330 bytes
Message:
OrderedDict([('correlation_id', '2017-02-02 01:14:29.555100'), ('message_id', '63622'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_f1d30800'), ('result', [[5, 0], [12, 1], [13, 1], [14, 1], [15, 0], [16, 1]]), ('sender', 'NodeMCU_f1d30800'), ('time_stamp', '2017-02-02 01:14:29.946100')])


[Result for request]:
___Request___:
{'correlation_id': '2017-02-02 01:14:29.555100', 'reply_to': 'Client_366', 'sender': 'Client_366', 'kwargs': {'pins': (5, 12, 13, 14, 15, 16)}, 'receiv

### Stop the demo

In [22]:
# Stopping
the_client.stop()
the_client = None
print ('\n[________________ Demo stopped ________________]\n')


[________________ Demo stopped ________________]

